In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("/kaggle/input/mse-2-ai-201-b-ai-d/train.csv")
test=pd.read_csv("/kaggle/input/mse-2-ai-201-b-ai-d/test.csv")

In [ ]:
test_id = test['id']

In [ ]:
test.describe()

In [ ]:
test = test.drop(columns=['id'])

In [ ]:
test.describe()

In [ ]:
col = train.columns
print(col)

In [ ]:
col_2 = test.columns
print(col_2)

In [ ]:
train.describe()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# BOXPLOT
numeric_cols = train.select_dtypes(include=['int64','float64']).columns
plt.figure(figsize=(18,12))
for i,col in enumerate(numeric_cols,1):
    plt.subplot(len(numeric_cols)//4+1,4,i)
    plt.boxplot(train[col].dropna())
    plt.title(col)
plt.tight_layout()
plt.show()


In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
X=train.drop(columns=['Class'])
y=train['Class']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
numeric_features=X.select_dtypes(include=['int64','float64']).columns
categorical_features=X.select_dtypes(include=['object']).columns

In [ ]:
# filling of numerical misisng values using mean strategy
numerical_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])
# filling of categorically misisng values using most frequent strategy
categorical_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('encode',OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Building up of ColumnTransformer
preprocessing=ColumnTransformer(transformers=[
    ('num',numerical_pipeline,numeric_features),
    ('cat',categorical_pipeline,categorical_features)
])

In [ ]:
model = GradientBoostingClassifier(
    n_estimators=920,
    learning_rate=0.02,
    max_depth=4,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.7,
    random_state=42
)

In [ ]:
pipeline=Pipeline(steps=[
    ('preprocessor',preprocessing),
    ('model',model)
])

In [ ]:
not_null_index = y_train.notnull()
X_train = X_train[not_null_index]
y_train = y_train[not_null_index]

In [ ]:
pipeline.fit(X_train,y_train)

In [ ]:
not_null_index = y_test.notnull()
X_test = X_test.loc[not_null_index]
y_test = y_test.loc[not_null_index]

In [ ]:
y_pred=pipeline.predict(X_test)

In [ ]:
accu=accuracy_score(y_pred,y_test)

In [ ]:
print(accu)

In [ ]:
y_final=pipeline.predict(test)

In [ ]:
submission=pd.DataFrame({
    'id':test_id,
    'Class':y_final
})

In [ ]:
submission.to_csv("Vedansh_Agarwal_202401100300276_submission_csv.csv",index=False)